In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm
import os

plt.style.use("seaborn-v0_8-whitegrid")
plt.rcParams["figure.figsize"] = (8,5)
plt.rcParams["lines.linewidth"] = 2

# load pre-scraped, cleaned data
vix_df = pd.read_csv("../data/raw/vix_ytd_ohlc.csv", parse_dates=["Date"])
spx_df = pd.read_csv("../data/raw/sp500_ytd_ohlcv.csv", parse_dates=["Date"])

# compute daily % changes
vix_df["dVIX"] = vix_df["Close"].pct_change()
spx_df["dSPX"] = spx_df["Adj Close"].pct_change()

merged = pd.merge(
    vix_df[["Date","dVIX"]],
    spx_df[["Date","dSPX"]],
    on="Date",
    how="inner"
).dropna()
merged.head()

In [ ]:
sns.set_palette("husl")

plt.figure(figsize=(7,6))
sns.regplot(
    data=merged,
    x="dSPX", y="dVIX",
    scatter_kws={"alpha":0.6, "s":25},
    line_kws={"color":"darkred"}
)
plt.title("Daily Relationship: ΔVIX vs S&P 500 Returns", fontsize=14, weight="bold")
plt.xlabel("S&P 500 Daily Return")
plt.ylabel("VIX Daily % Change")

plt.tight_layout()
os.makedirs("../reports/figures", exist_ok=True)
plt.savefig("../reports/figures/vix_spx_regression.png", dpi=300, bbox_inches="tight")
plt.show()


In [ ]:
X = sm.add_constant(merged["dSPX"])   # add intercept
y = merged["dVIX"]

model = sm.OLS(y, X).fit()
print(model.summary())